In this exercise, we will adopt UNet for the Image inpainting problem

# Import libs

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torcheval.metrics.functional import peak_signal_noise_ratio

# Data

We will use the same dataset in Ex2, but with additional mask on the data

In [3]:
class ImageDataset(Dataset):
    def __init__(self, img_dir, transforms=None):
        self.img_dir = img_dir
        self.transforms = transforms
        self.imgs = os.listdir(os.path.join(img_dir, 'high_res'))

    def __len__(self):
        return len(self.imgs)
    
    def create_mask(self, img):
        # Create a mask with random 1 and 0 values to hide some parts of the image
        mask = img.copy()
        mask[np.random.random(img.shape) < 0.2] = 0
        return mask
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, 'high_res', self.imgs[idx])
        
        target_image = Image.open(img_path).convert("RGB")
        input_image = self.create_mask(np.array(target_image))
        input_image = Image.fromarray(input_image)

        if self.transforms:
            input_image = self.transforms(input_image)
            target_image = self.transforms(target_image)
        
        return input_image, target_image

# Modeling

In [6]:
import timm

ValueError: mutable default <class 'timm.models.maxxvit.MaxxVitConvCfg'> for field conv_cfg is not allowed: use default_factory